In [ ]:
#%load_ext autotime

SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'TINYAES128C'  
num_traces = 300000
CHECK_CORR = False



In [ ]:

#%load_ext autotime

SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'
CRYPTO_TARGET = 'TINYAES128C'  
num_traces = 200000
CHECK_CORR = False


In [ ]:
%run jupyter/Setup_Scripts/Setup_Generic.ipynb

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd hardware/victims/firmware/simpleserial-aes

make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3

In [ ]:
cmd = "hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, cmd)
print(cmd)

In [ ]:
scope.clock.adc_rate

# Set Gain

In [ ]:
scope.gain.gain = 55
# gain for em 69
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()
key, text = ktp.next()
target.set_key(key)

scope.adc.offset = 0
scope.adc.samples = 5000
ktp = cw.ktp.Basic()

# Key Setup

In [ ]:
dir(ktp)
aaa = ktp.getInitialKey()
#print(aaa)
bbb = ktp.getKeyType()
print(bbb)

# The appropriate key can be uncommented based on the sspecific dataset,
# note that the default key is automatically set : We reffer to the default key as K1
#
# Here are the Different Key options that we use in data collection: 


# K2
# key_str = 'aa,80,d8,a7,84,d3,3f,5c,0b,90,a9,85,20,8e,ff,4a'

# K3
# key_str = 'd2,d5,01,68,82,83,91,43,96,9e,e9,a2,53,a7,52,e1'

# K4
# key_str = 'e6,de,35,a9,a5,23,19,df,c6,cc,bb,ba,c1,36,c3,bf'


# ktp.fixed_key = False
# ktp.setInitialKey(key_str)
ktp.fixed_key = True
key, text = ktp.next()

# SAMPLE KEY AND TXT BOX

In [ ]:
target.set_key(key)
for i in range(3):
    key, text = ktp.next()
    print(key)
    print(text)
    print('==============')

In [ ]:
key

In [ ]:
%run "/home/ucsec/chipwhisperer/jupyter/Helper_Scripts/plot.ipynb"
plot = real_time_plot(plot_len=5000)

In [ ]:
import numpy as np
import chipwhisperer as cw
import chipwhisperer.analyzer as cwa

#project = cw.create_project("projects/hfield_demo.cwp", overwrite = True)

from tqdm import tnrange
import time
## define val to store data (trace,text,key)
power_trace, plain_text = [],[]

for i in tnrange(num_traces, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    
    power_trace.append(np.array(trace[0],dtype=np.float32))
    plain_text.append(np.array(trace[1]))
    
    ## send trace for visualization every 25th trace
    if i % 25 == 0:
        plot.send(trace)
    #time.sleep(.05)
    
key2=key        
key = np.array(key)  ## as key in all iterations are the same
power_trace = np.array(power_trace,dtype=np.float32)
plain_text = np.array(plain_text)

# Scope Specs

In [ ]:
print("Capture rate: "+str
      (scope.clock.adc_freq/10e5)+"MHz")

# Save the data

In [ ]:
output_path='/home/ucsec/Desktop/'
os.makedirs(output_path, exist_ok=True)

file_name='val_diff_key.npz'

outpath = os.path.join(output_path,file_name)
np.savez(outpath, power_trace=trace_array, plain_text=textin_array,key=key)


In [ ]:
## disconnect device
scope.dis()
target.dis()

In [ ]:
hex_key = [hex(val) for val in key]
print(hex_key)